In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('data.xlsx', header=None)

In [3]:
data = data.replace("Legend:", None)

In [4]:
header_rows = data[(data[1] == 'Date') | (data[1] == 'Dates')].copy()

In [5]:
header_rows[0] = 'PGY'
header_rows[1] = 'Dates'

Make the data tidy

In [6]:
ll = list()
for (start, cols), end in zip(header_rows.iterrows(), header_rows.index[1:].append(data.index[-1:])):
    d = data.loc[start+1:end, :cols.last_valid_index()]
    
    d = d[~d[0].isna()]
    
    d.columns = cols.iloc[:d.shape[1]]
    
    d = d.rename({'Date': 'Name', 'Dates': 'Name'}, axis=1)
    
    ll.append(d)

In [7]:
ll[0]

,PGY,Name,July,August,September,October,November,December,January,February,March,April,May,June
1,5,Akateh,ACS,NaN,Zollinger,NaN,East Gen Surg,NaN,Ellison,NaN,CRS,NaN,HPB,NaN
2,5,Koller,HPB,NaN,ACS,NaN,Zollinger,NaN,East Gen Surg,NaN,Ellison,NaN,CRS,NaN
3,5,Guzman,CRS,NaN,HPB,NaN,ACS,NaN,Zollinger,NaN,East Gen Surg,NaN,Ellison,NaN
4,5,Drews,Ellison,NaN,CRS,NaN,HPB,NaN,ACS,NaN,Zollinger,NaN,East Gen Surg,NaN
5,5,Horwood,East Gen Surg,NaN,Ellison,NaN,CRS,NaN,HPB,NaN,ACS,NaN,Zollinger,NaN
6,5,Ricci,Zollinger,NaN,East Gen Surg,NaN,Ellison,NaN,CRS,NaN,HPB,NaN,ACS,NaN


In [8]:
def unpivot(df):
    return df.ffill(axis=1).melt(id_vars=['Name', 'PGY'], value_name='Rotation', var_name='Date')

tidy_data = pd.concat([unpivot(df) for df in ll])

Deal with dates

In [9]:
header_rows.iloc[:, 2:]

,2,3,4,5,6,7,8,9,10,11,12,13,14
0,July,August,September,October,November,December,January,February,March,April,May,June,NaN
10,July 1 - Aug. 22,Aug. 23 - Oct. 17,Oct. 18 - Dec. 12,Dec. 13 - Feb. 6,Feb. 7 - Mar. 27,Mar. 28 - May 15,May 16 - June 30,NaN,NaN,NaN,NaN,NaN,NaN
19,July 1 - Aug. 22,Aug. 23 - Oct. 17,Oct. 18 - Dec. 12,Dec. 13 - Feb. 6,Feb. 7 - Mar. 27,Mar. 28 - May 15,May 16 - June 30,NaN,NaN,NaN,NaN,NaN,NaN
28,7/1-7/26,7/27-8/23,8/24-9/20,9/21-10/18,10/19-11/15,11/16-12/13,12/14-1/10,1/11-2/7,2/8-3/7,3/8-4/4,4/5-5/2,5/3-5/30,5/31-6/30
42,7/1-7/26,7/27-8/23,8/24-9/20,9/21-10/18,10/19-11/15,11/16-12/13,12/14-1/10,1/11-2/7,2/8-3/7,3/8-4/4,4/5-5/2,5/3-5/30,5/31-6/30


In [10]:
# For PGY 5, just use date range

pgy5_dates = pd.DataFrame({
    'Date': header_rows.iloc[0, 2:14],
    'start_date': pd.date_range('July 1, 2020', 'June 1, 2021', freq='MS'),
    'end_date': pd.date_range('July 1, 2020', 'July 1, 2021', freq='M'),
})

pgy5_dates

,Date,start_date,end_date
2,July,2020-07-01,2020-07-31
3,August,2020-08-01,2020-08-31
4,September,2020-09-01,2020-09-30
5,October,2020-10-01,2020-10-31
6,November,2020-11-01,2020-11-30
7,December,2020-12-01,2020-12-31
8,January,2021-01-01,2021-01-31
9,February,2021-02-01,2021-02-28
10,March,2021-03-01,2021-03-31
11,April,2021-04-01,2021-04-30


In [11]:
# For the others... use tricks
def make_range(s):
    result = pd.to_datetime([t + ', 2020' for t in s.split(' - ')]).to_list()
    
    result = [r if r >= pd.Timestamp(2020, 7, 1) else r + pd.DateOffset(years=1) for r in result]
    
    return result

pgy43_dates = pd.DataFrame(header_rows.iloc[1, 2:9].apply(make_range).to_list())
pgy43_dates.columns = ['start_date', 'end_date']
pgy43_dates['Date'] = header_rows.iloc[1, 2:9].reset_index(drop=True)

pgy43_dates

,start_date,end_date,Date
0,2020-07-01,2020-08-22,July 1 - Aug. 22
1,2020-08-23,2020-10-17,Aug. 23 - Oct. 17
2,2020-10-18,2020-12-12,Oct. 18 - Dec. 12
3,2020-12-13,2021-02-06,Dec. 13 - Feb. 6
4,2021-02-07,2021-03-27,Feb. 7 - Mar. 27
5,2021-03-28,2021-05-15,Mar. 28 - May 15
6,2021-05-16,2021-06-30,May 16 - June 30


In [12]:
# For the others... use tricks
def make_range(s):
    result = pd.to_datetime([t + '/2020' for t in s.split('-')]).to_list()
    
    result = [r if r >= pd.Timestamp(2020, 7, 1) else r + pd.DateOffset(years=1) for r in result]
    
    return result

pgy12_dates = pd.DataFrame(header_rows.iloc[3, 2:].apply(make_range).to_list())
pgy12_dates.columns = ['start_date', 'end_date']
pgy12_dates['Date'] = header_rows.iloc[3, 2:].reset_index(drop=True)

pgy12_dates

,start_date,end_date,Date
0,2020-07-01,2020-07-26,7/1-7/26
1,2020-07-27,2020-08-23,7/27-8/23
2,2020-08-24,2020-09-20,8/24-9/20
3,2020-09-21,2020-10-18,9/21-10/18
4,2020-10-19,2020-11-15,10/19-11/15
5,2020-11-16,2020-12-13,11/16-12/13
6,2020-12-14,2021-01-10,12/14-1/10
7,2021-01-11,2021-02-07,1/11-2/7
8,2021-02-08,2021-03-07,2/8-3/7
9,2021-03-08,2021-04-04,3/8-4/4


In [13]:
all_dates = pd.concat([pgy5_dates, pgy43_dates, pgy12_dates])

In [14]:
final_clean = pd.merge(tidy_data, all_dates, on='Date').drop('Date', axis=1)

Output to database

In [15]:
import sqlalchemy

engine = sqlalchemy.create_engine('sqlite:///resident_schedule.db', echo=True)
conn = engine.connect()


final_clean.to_sql('schedule', conn)

2020-07-24 18:09:08,876 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-24 18:09:08,877 INFO sqlalchemy.engine.base.Engine ()
2020-07-24 18:09:08,878 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-24 18:09:08,878 INFO sqlalchemy.engine.base.Engine ()
2020-07-24 18:09:08,884 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("schedule")
2020-07-24 18:09:08,884 INFO sqlalchemy.engine.base.Engine ()
2020-07-24 18:09:08,886 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("schedule")
2020-07-24 18:09:08,887 INFO sqlalchemy.engine.base.Engine ()
2020-07-24 18:09:08,889 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE schedule (
	"index" BIGINT, 
	"Name" TEXT, 
	"PGY" TEXT, 
	"Rotation" TEXT, 
	start_date DATETIME, 
	end_date DATETIME
)


2020-07-24 18:09:08,890 INFO sqlalchemy.engine.base.Engine ()
2020-07-24 18:09:08,895 INFO sqlalchemy.engine.base.Engine COMMIT
20